In [40]:
import numpy as np
import pandas as pd 
from itertools import izip

Offensive Rating is defined as the team points scored per 100 possessions while the player is on the court.

Defensive Rating is defined as the number of points per 100 possessions that the team alows while that individual player is on the court. 

A possession is ended by
 -1 made field goal attempts
 -2 Made final free throw attempt
 -3 Missed final free throw attempt that results in a defensive reboud
 -4 Missed field goal attempt that resutls in a defensive rebound
 -5 turnover
 -6 end of time period

In [41]:
EventCodes = pd.read_csv('Event_Codes.txt', sep='\t')
#Strip excess whitespace that I noticed in Event Message Type Description
EventCodes['Event_Msg_Type_Description'] = EventCodes['Event_Msg_Type_Description'].map(str.strip)
#eventCodes['Action_Type_Description'] = eventCodes['Action_Type_Description'].map(str.strip)

GameLineup = pd.read_csv('Game_Lineup.txt', sep='\t')
PlayByPlay = pd.read_csv('Play_by_Play.txt', sep='\t')

In [42]:
from collections import defaultdict

In [43]:
EventScoringValues = defaultdict(dict) #the point value of a play, for the offensive team
EventPossChange= defaultdict(dict) #whether the team could constitute a change of possesion
EventSub = defaultdict(dict)

In [44]:
for row in EventCodes.iterrows():
    #initialize
    e_type, a_type = row[1]['Event_Msg_Type'], row[1]['Action_Type']
    EventScoringValues[e_type][a_type] = 0
    EventPossChange[e_type][a_type] = False
    EventSub[e_type][a_type] = False

1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
2 50
2 51
2 52
2 53
2 54
2 55
2 56
2 57
2 58
2 59
2 60
2 61
2 62
2 63
2 64
2 65
2 66
2 67
2 71
2 72
2 73
2 74
2 75
2 76
2 77
2 78
2 79
2 80
2 81
2 82
2 83
2 84
2 85
2 86
2 87
2 88
2 89
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
2 100
2 101
2 102
2 103
2 104
2 105
2 106
2 107
2 108
2 109
2 110
3 0
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 25
3 26
3 27
3 28
3 29
4 0
4 1
4 2
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 

In [46]:
_,idx1 = np.unique(EventCodes['Event_Msg_Type'], return_index=True)
_,idx2 = np.unique(EventCodes['Action_Type'], return_index=True)

In [47]:
idx = np.array(list(set(idx1).union(idx2)))
idx.sort()

In [48]:
for i in idx:
    print EventCodes['Event_Msg_Type'][i], EventCodes['Event_Msg_Type_Description'][i],EventCodes['Action_Type'][i], EventCodes['Action_Type_Description'][i]

1 Made Shot 0 No Shot
1 Made Shot 1 Jump Shot
1 Made Shot 2 Running Jump Shot
1 Made Shot 3 Hook Shot
1 Made Shot 4 Tip Shot
1 Made Shot 5 Layup Shot
1 Made Shot 6 Driving Layup Shot
1 Made Shot 7 Dunk Shot
1 Made Shot 8 Slam Dunk Shot
1 Made Shot 9 Driving Dunk Shot
1 Made Shot 40 Layup Shot
1 Made Shot 41 Running Layup Shot
1 Made Shot 42 Driving Layup Shot
1 Made Shot 43 Alley Oop Layup shot
1 Made Shot 44 Reverse Layup Shot
1 Made Shot 45 Jump Shot
1 Made Shot 46 Running Jump Shot
1 Made Shot 47 Turnaround Jump Shot
1 Made Shot 48 Dunk Shot
1 Made Shot 49 Driving Dunk Shot
1 Made Shot 50 Running Dunk Shot
1 Made Shot 51 Reverse Dunk Shot
1 Made Shot 52 Alley Oop Dunk Shot
1 Made Shot 53 Tip Shot
1 Made Shot 54 Running Tip Shot
1 Made Shot 55 Hook Shot
1 Made Shot 56 Running Hook Shot
1 Made Shot 57 Driving Hook Shot
1 Made Shot 58 Turnaround Hook Shot
1 Made Shot 59 Finger Roll Shot
1 Made Shot 60 Running Finger Roll Shot
1 Made Shot 61 Driving Finger Roll Shot
1 Made Shot 62 Turna

In [49]:
#Hardcore the scoring values of points
for action in EventScoringValues[1]:
    EventScoringValues[1][action] = 2 # i think some of these will be to be 3's
    
for action in EventScoringValues[3]:
    # are all free throw worth 1? technical?
    EventScoringValues[3][action] = 1
    
#all others 0

In [51]:
for action in EventPossChange[1]:
    EventPossChange[1][action] = True

    
free_throws = EventCodes['Event_Msg_Type'] == 3
for action in EventPossChange[3]:
    same_action_code = EventCodes['Action_Type'] == action
    
    split_desc= str(EventCodes[np.logical_and(free_throws, same_action_code)]['Action_Type_Description'].iloc[0]).split('of')
    
    if len(split_desc) == 2:
        #check if last shot 
        shot,tot =  int(split_desc[0].strip()[-1]), int(split_desc[1].strip())
        if shot==tot: #last shot
            EventPossChange[3][action] = True
        
for action in EventPossChange[4]:
    EventPossChange[4] = None # all rebounds have to be checked
    
    
for action in EventPossChange[5]:
    EventPossChange[5] = True
    
for action in EventPossChange[10]:
    EventPossChange[5] = None # same with jump balls

### Order of operations
For Game in Games:

    For possession in Game:
        for player in possession:
            update scoring
    normalize by count

In [62]:
class Game(object):
    
    def __init__(self, game_id, game_lineup, play_by_play):
        
        self.game_id = game_id
        self.play_by_play = self.chunk_pbp(play_by_play[play_by_play['Game_id'] == game_id])
        
        # tuple of lists of each player on the team
        self.teams = self.make_teams(game_lineup[game_lineup['Game_id']==game_id])
        self.poss_team_0 = None
    
    def score_event(self, event):
        
        ifself.poss_team_0 is None:
            pass # possession has not been established, no scoring plays will have occurred
        for play in event:
            play_e, play_a = event[2], event[6]
            play_value = EventScoringValues[play_e][play_a]
            
            for team, offense_or_defense in zip(self.teams, (self.poss_team_0, not self.poss_team_0)):
                
        
    def possession_change(self, event):
        pass
    
    def chunk_pbp(self, game_pbp):
        sorted_pbp = self.sort_pbp(game_pbp)
        real_time = sorted_pbp['PC_Time'].as_matrix()+  (4-sorted_pbp['Period'].as_matrix())*7200
        _, unique_idxs = np.unique(real_time, return_index=True)
        return np.split(game_pbp.as_matrix(), indices_or_sections=unique_idxs[::-1] )[1:] # remove the first one thats empty
        
    def sort_pbp(self, game_pbp):
        return game_pbp.sort_values(['Period', 'PC_Time', 'WC_Time', 'Event_Num'],\
                                    ascending=[True, False, True,True])
    
    def __iter__(self):
        return self.play_by_play.__iter__()
    
    def __next__(self):
        return self.__next__()
    
    def make_teams(self, game_lineup):
        team1_id, team2_id = np.unique(game_lineup['Team_id'])
        team1, team2 = {},{}
        for team_id, team_arr in izip((team1_id, team2_id), (team1, team2)):
            players_on_team = game_lineup[game_lineup['Team_id']==team_id]['Person_id'].unique()
            
            for player in players_on_team:
                player_data = game_lineup[game_lineup['Person_id'] == player].iloc[0] #ignore possessions
                team_arr[player] = Player(player_data)
                
        return team1, team2

In [63]:
class Player(object):
    
    def __init__(self, player_data):
        self.player_id = player_data['Person_id']
        self.team_id = player_data['Team_id']
        self.active = player_data['status'] == 'A'
        
        self.off_points = 0
        self.def_points = 0
        
        self.off_pos = 0
        self.def_pos = 0

In [64]:
game0_id = GameLineup.iloc[0]['Game_id']
game0 = Game(game0_id, GameLineup, PlayByPlay)

In [65]:
for event in game0:
    print len(event), event

2 [['006728e4c10e957011e1f24878e6054a' 2 12 1 546410 7200 0 0 0 0
  '1473d70e5646a26de3c52aa1abd85b1f' '0370a0d090da0d0edc6319f120187e0e'
  '0370a0d090da0d0edc6319f120187e0e' '0370a0d090da0d0edc6319f120187e0e' 0
  0 0 0]
 ['006728e4c10e957011e1f24878e6054a' 4 10 1 546420 7200 0 0 0 0
  '01be0ad4af7aeb1f6d2cc2b6b6d6d811' '8d2127290c94bd41b82a2938734bc750'
  '99104de2626f67c1fa2ce70504970c3f' '766802a8fda500d7945950de7398c9c6' 2
  4 5 4]]
3 [['006728e4c10e957011e1f24878e6054a' 7 1 1 546590 7020 79 2 0 0
  '01be0ad4af7aeb1f6d2cc2b6b6d6d811' '42e0d7167f04a4ff958c6442da0e6851'
  '0370a0d090da0d0edc6319f120187e0e' '0370a0d090da0d0edc6319f120187e0e' 2
  4 0 0]
 ['006728e4c10e957011e1f24878e6054a' 8 6 1 546620 7020 2 0 0 1
  '01be0ad4af7aeb1f6d2cc2b6b6d6d811' '616281dee946056b071699476fdee9ec'
  '42e0d7167f04a4ff958c6442da0e6851' '0b1c631effab25d4af2d16fc123c3d31' 2
  5 4 1]
 ['006728e4c10e957011e1f24878e6054a' 10 3 1 546810 7020 10 1 0 0
  '01be0ad4af7aeb1f6d2cc2b6b6d6d811' '42e0d7167f04a4ff9